In [4]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D

import data_util as du
import performance_util as pu
import importlib
from PCA import PCA
import kmeans
import softmax_logit as logsoft

%matplotlib inline

In [5]:
data_dir = './data/Notes/new_mnist/'
importlib.reload(du)

<module 'data_util' from 'E:\\7th_Sem\\Machine Learning\\Assignments\\Assignment_2\\src\\data_util.py'>

In [6]:
data_dir2 = './data/mnist/'
random_seed = 1
np.set_printoptions(precision=4)

In [8]:
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels


In [9]:
X_train, Y_train = load_mnist(data_dir2)
Y_train = Y_train.reshape(len(Y_train) , 1)

#X_test, Y_test = du.load_fashion_mnist(data_dir + 't10k')

print(X_train.shape)
print(Y_train.shape)

X_test, Y_test = du.load_mnistx(data_dir,10)
Y_test = Y_test.reshape(len(Y_test), 1)


print(X_test.shape)
print(Y_test.shape)

(60000, 784)
(60000, 1)
(7947, 784)
(7947, 1)


In [11]:
#X_train, Y_train, X_test , Y_test = du.split_data(X_train, Y_train, 0.66)

In [10]:
X_train = du.normalize2(X_train)
X_test = du.normalize2(X_test)

In [11]:
print(X_train.shape)
print(Y_train.shape)

print(X_test.shape)
print(Y_test.shape)

(60000, 784)
(60000, 1)
(7947, 784)
(7947, 1)


In [12]:
pca = PCA(n_components=100)
pca.fit(X_train)

X_train_reduced_1 = pca.transform(X_train)
X_test_reduced_1 = pca.transform(X_test)

print(np.sum(pca.explained_variance_ratio_))


print(X_train_reduced_1.shape)
print(X_test_reduced_1.shape)

0.9078218541788241
(60000, 100)
(7947, 100)


In [16]:
logist= logsoft.softmaxLogit( iterations = 80000, Lambda= 0.0, alpha = 0.015, num_classes =10)

logist.fit(X_train_reduced_1, Y_train)

In [18]:
result= logist.predict(X_test_reduced_1)

result = result.reshape((len(X_test_reduced_1),))
Y_test = Y_test.reshape((len(X_test_reduced_1),))

#np.savetxt("foo.csv", a, delimiter=",")

print( "Accuracy:" , pu.model_accuracy(result, Y_test))
print( "macro_average: ", pu.model_macro_average(result, Y_test, 10))
print("micro_average: ", pu.model_micro_average(result, Y_test, 10))
print( "f1: ", pu.model_f1(result, Y_test,10))
print("precision: ", pu.model_precision(result, Y_test,10))
print("recall:", pu.model_recall(result, Y_test,10))


Accuracy: 0.4373977601610671
macro_average:  [0.4081 0.4335 0.4204]
micro_average:  [0.4374 0.4374 0.4374]
f1:  [[0.7281]
 [0.5566]
 [0.4213]
 [0.6192]
 [0.4046]
 [0.2574]
 [0.2855]
 [0.152 ]
 [0.5082]
 [0.1674]]
precision:  [[0.6148]
 [0.4686]
 [0.5069]
 [0.5621]
 [0.3904]
 [0.2997]
 [0.3732]
 [0.1584]
 [0.4685]
 [0.2384]]
recall: [[0.8926]
 [0.6853]
 [0.3604]
 [0.6893]
 [0.4199]
 [0.2256]
 [0.2312]
 [0.1461]
 [0.5553]
 [0.1289]]


In [14]:
print(result.shape)

(7947,)


In [19]:
np.savetxt("foo.csv", Y_test, delimiter=",")

In [16]:
from sklearn.svm import SVC # "Support Vector Classifier" 
clf = SVC(kernel='poly', degree= 4) 
  
# fitting x samples and y classes 
clf.fit(X_train_reduced_1, Y_train)

result = clf.predict(X_test_reduced_1) 

result = result.reshape((len(X_test_reduced_1),))
Y_test = Y_test.reshape((len(X_test_reduced_1),))
#np.savetxt("foo.csv", Y_test, delimiter=",")

print( "Accuracy:" , pu.model_accuracy(result, Y_test))

D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Accuracy: 0.10557443060274317


In [13]:
print(result.shape)
print( "Accuracy:" , pu.model_accuracy(result, Y_test))

(7947,)
Accuracy: 0.10557443060274317


In [ ]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='lbfgs', alpha=1e-4,
                    hidden_layer_sizes=(5, 2), random_state=1)

clf.fit(X_train, Y_train)

In [18]:
import tensorflow as tf

In [30]:
x = tf.placeholder(tf.float32, [0,784])
y = tf.placeholder(tf.float32, [0, 10])



W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))


In [36]:
with tf.name_scope( "Wx_b") as scope:
    model= tf.nn.softmax(tf.matmul(x,W)+b)

w_h = tf.summary.histogram("weights", W)
b_h = tf.summary.histogram("bias", b)  

In [38]:
with tf.name_scope( "cost_func") as scope:
    cost= tf.reduce_sum(y*tf.log(model))
    tf.summary.scalar("cost_func", cost)

In [41]:
with tf.name_scope( "GD") as scope:
    GD = tf.train.GradientDescentOptimizer(0.05).minimize(cost)
    

In [46]:
init = tf.global_variables_initializer()
merged_summary_operator = tf.summary.merge_all()


ModuleNotFoundError: No module named 'input_data'

In [54]:
with tf.Session() as session:
    session.run(init)
    
    summary_writer = tf.summary.FileWriter('./logs', graph = session.graph)
    for itr in range(1000):
        batch_xs, batch_ys = mnist_train.next_batch(batch_size)
        session.run(GD, feed_dict= {x: batch_xs, y:batch_ys})
        
        avg_cost = session.run(cost_func, feed_dict = {x:batch_xs, y:batch_ys})
        
        summary_str = session.run(merged_summary_operator, feed_dict={x:batch_xs, y:batch_ys})
        

NameError: name 'mnist_train' is not defined

In [ ]:
session.run(train)
print(session.run(w))

In [26]:
for i in range(1000):
    session.run(train)
    print(session.run(w))
    

1.8
2.44
2.9520001
3.3616002
3.68928
3.9514241
4.1611395
4.328912
4.4631295
4.5705037
4.656403
4.7251225
4.780098
4.8240786
4.859263
4.88741
4.9099283
4.9279428
4.942354
4.953883
4.9631066
4.970485
4.976388
4.9811106
4.9848886
4.9879107
4.990329
4.992263
4.99381
4.995048
4.9960384
4.996831
4.9974647
4.9979715
4.9983773
4.998702
4.9989614
4.9991693
4.9993353
4.9994683
4.9995747
4.9996595
4.9997277
4.999782
4.9998255
4.9998603
4.9998884
4.999911
4.9999285
4.999943
4.999954
4.9999633
4.9999704
4.999976
4.999981
4.9999847
4.9999876
4.99999
4.999992
4.9999933
4.9999948
4.9999957
4.9999967
4.999997
4.9999976
4.999998
4.9999986
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.

4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999
4.999999


In [13]:
data_dirx = 'C:/Users/rupes/OneDrive/Pictures/Screenshots/'
X_test_1, Y_test_1 = du.load_mnistx(data_dirx,1)
Y_test_1 = Y_test_1.reshape(len(Y_test_1), 1)

ValueError: cannot reshape array of size 4196352 into shape (784,)

In [11]:
print(X_test_1.shape)

(0,)
